# Import libraries

In [340]:
# uncomment to install libraries
# ! pip install numpy pandas matplotlib seaborn requests selenium bs4
# ! pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import selenium
from bs4 import BeautifulSoup
import time

# Test web scraping escape room reviews

## World of Escapes map page URL

In [341]:
map_page_url = "https://worldofescapes.com/map"

# also home page
home_page_url = "https://worldofescapes.com"

## Map page response

In [342]:
map_page_response = requests.get(map_page_url)

## Map page soup

In [343]:
map_page_soup = BeautifulSoup(map_page_response.text, "html.parser")

## Map states and city href and URLs

In [344]:
# dictionary container for states and their city href and urls
state_city_href = {}
state_city_urls = {}

# loop through states and add their city href
for state_item_i in map_page_soup.find_all("div", {"class": "col-lg-3 col-md-4 col-sm-6 col-xs-6 state-item"}):
    # state str
    state_i = state_item_i.find("h3").get_text().strip().lower()
    
    # add states to href and url dictionaries
    state_city_href[state_i] = []
    state_city_urls[state_i] = []
    
    # add cities to state values
    for city_item_j in state_item_i.find_all("li", {"class": "city-item"}):
        state_city_href[state_i].append(city_item_j.a["href"])
        state_city_urls[state_i].append(home_page_url + city_item_j.a["href"])

# See URLs (which incorporate href)
state_city_urls

{'alabama': ['https://worldofescapes.com/auburn',
  'https://worldofescapes.com/birm',
  'https://worldofescapes.com/daleville',
  'https://worldofescapes.com/dothan',
  'https://worldofescapes.com/florence-al',
  'https://worldofescapes.com/gadsden',
  'https://worldofescapes.com/henagar',
  'https://worldofescapes.com/huntsville',
  'https://worldofescapes.com/mobile',
  'https://worldofescapes.com/montgomery',
  'https://worldofescapes.com/gulf-shores',
  'https://worldofescapes.com/oxford-al',
  'https://worldofescapes.com/tuscaloosa'],
 'alaska': ['https://worldofescapes.com/anchorage',
  'https://worldofescapes.com/fairbanks',
  'https://worldofescapes.com/juneau',
  'https://worldofescapes.com/skagway'],
 'arizona': ['https://worldofescapes.com/cottonwood',
  'https://worldofescapes.com/flagstaff',
  'https://worldofescapes.com/lake-havasu-city',
  'https://worldofescapes.com/phoenix',
  'https://worldofescapes.com/prescott',
  'https://worldofescapes.com/tucson',
  'https://wor

## Map states and city page responses and soups

In [345]:
# containers for state and their city page responses and soups
state_city_responses = {}
state_city_soups = {}

# loop through responses and soups
# just alabama and alaska for now
for state_key, city_url in {state: state_city_urls[state] for state in ("alabama", "alaska", "arizona", "arkansas")}.items():
    # add state key to dictionary containers
    state_city_responses[state_key] = []
    state_city_soups[state_key] = []
    
    for url in city_url:
        # get response
        response_j = requests.get(url)
        
        # make soup
        soup_j = BeautifulSoup(response_j.text, "html.parser")
        
        # append response and soup to state
        state_city_responses[state_key].append(response_j)
        state_city_soups[state_key].append(soup_j)
        
        # sleep for a random number of seconds
        sleep_time_j = np.random.randint(low = 1, high = 5, size = 1)
        time.sleep(sleep_time_j)
        
        # print loop summary
        print("Made soup for {} in {}. Sleeping for {} seconds...".format(url, state_key, sleep_time_j))

Made soup for https://worldofescapes.com/auburn in alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/birm in alabama. Sleeping for [3] seconds...
Made soup for https://worldofescapes.com/daleville in alabama. Sleeping for [1] seconds...
Made soup for https://worldofescapes.com/dothan in alabama. Sleeping for [1] seconds...
Made soup for https://worldofescapes.com/florence-al in alabama. Sleeping for [2] seconds...
Made soup for https://worldofescapes.com/gadsden in alabama. Sleeping for [2] seconds...
Made soup for https://worldofescapes.com/henagar in alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/huntsville in alabama. Sleeping for [2] seconds...
Made soup for https://worldofescapes.com/mobile in alabama. Sleeping for [3] seconds...
Made soup for https://worldofescapes.com/montgomery in alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/gulf-shores in alabama. Sleeping for [3] seconds...
Made soup f

## Escape room href and URLs

In [346]:
# container for city urls within states
state_city_room_href = {}
state_city_room_urls = {}

# loop through states and cities within states
# just alabama and alaska for now (state_city_soups only contains soups from alabama and alaska from previous code chunk)
for state_key, city_soup in state_city_soups.items():
    # add state key to dictionary containers
    state_city_room_href[state_key] = {}
    state_city_room_urls[state_key] = {}

    for j, soup in enumerate(city_soup):
        # add dictionary container to each city href value
        # nested dictionaries, e.g., {"alabama": "/auburn": [NEW escape room href go here, ...]}
        state_city_room_href[state_key][state_city_href[state_key][j]] = []
        state_city_room_urls[state_key][state_city_href[state_key][j]] = []
        
        # loop through a tags with escape room href
        if soup.find("div", {"data-content": "other-quests"}) is not None:
            for a in soup.find("div", {"data-content": "other-quests"}).find_all("a", {"class": "item-hover quest_tile_hover_link"}):
                
                # add href
                state_city_room_href[state_key][state_city_href[state_key][j]].append(a["href"])
                
                # add url
                state_city_room_urls[state_key][state_city_href[state_key][j]].append(home_page_url + a["href"])
                
                # print loop summary
                print("Added {} for {}, {}.".format(home_page_url + a["href"], state_city_href[state_key][j], state_key))
                
        else:
            print("None")

Added https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-cabin for /auburn, alabama.
Added https://worldofescapes.com/auburn/quests/auburn-escape-zones-imprisoned for /auburn, alabama.
Added https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-puzzler-vs-superheroes for /auburn, alabama.
Added https://worldofescapes.com/auburn/quests/auburn-escape-zones-vault for /auburn, alabama.
Added https://worldofescapes.com/auburn/quests/auburn-escape-zones-black-beards-brig for /auburn, alabama.
Added https://worldofescapes.com/birm/quests/Locked-In-THE-LABORATORY for /birm, alabama.
Added https://worldofescapes.com/birm/quests/locked-in-birm-the-classroom for /birm, alabama.
Added https://worldofescapes.com/birm/quests/Locked-In-THE-WAREHOUSE for /birm, alabama.
Added https://worldofescapes.com/birm/quests/nightmare-at-3008-3008-house for /birm, alabama.
Added https://worldofescapes.com/birm/quests/nightmare-at-3008-camp-mary-lee for /birm, alabama.
Added https://world

Added https://worldofescapes.com/phoenix/quests/escape-games-az-blaines-basement for /phoenix, arizona.
Added https://worldofescapes.com/phoenix/quests/dare-to-escape-az-the-cell for /phoenix, arizona.
Added https://worldofescapes.com/phoenix/quests/lockdown-escape-rooms-samurai for /phoenix, arizona.
Added https://worldofescapes.com/phoenix/quests/the-study-az for /phoenix, arizona.
Added https://worldofescapes.com/phoenix/quests/escape-zone-the-inheritance for /phoenix, arizona.
Added https://worldofescapes.com/phoenix/quests/escape-rooms-mesa-the-legend-of-the-lost-dutchman for /phoenix, arizona.
Added https://worldofescapes.com/phoenix/quests/Epic-Escape-Game-The-Fortune-Teller for /phoenix, arizona.
Added https://worldofescapes.com/phoenix/quests/north-valley-escape-room-escape-from-alcatraz for /phoenix, arizona.
Added https://worldofescapes.com/phoenix/quests/phoenix-escape-room-gilbert-pipe-works for /phoenix, arizona.
Added https://worldofescapes.com/phoenix/quests/paradox-esc

Added https://worldofescapes.com/tucson/quests/escape-room-tucson-area-51 for /tucson, arizona.
Added https://worldofescapes.com/tucson/quests/ace-of-escape-international-thief for /tucson, arizona.
Added https://worldofescapes.com/tucson/quests/fox-in-a-box-tucson-central-bank for /tucson, arizona.
Added https://worldofescapes.com/tucson/quests/escape-room-tucson-red-rum-cottage for /tucson, arizona.
Added https://worldofescapes.com/tucson/quests/down-the-rabbit-hole-the-asylum for /tucson, arizona.
Added https://worldofescapes.com/tucson/quests/down-the-rabbit-hole-ante-up-room for /tucson, arizona.
Added https://worldofescapes.com/tucson/quests/ace-of-escape-bad-medicine for /tucson, arizona.
Added https://worldofescapes.com/tucson/quests/escape-room-tucson-inside-job for /tucson, arizona.
Added https://worldofescapes.com/tucson/quests/will-you-escape-serial-killer-room for /tucson, arizona.
Added https://worldofescapes.com/tucson/quests/ace-of-escape-beat-the-clock for /tucson, ari

## Escape room page responses and soups

In [348]:
# container for escape room page responses and urls
state_city_room_responses = {}
state_city_room_soups = {}

# loop through city href nested within states
# just alabama and alaska for now (state_city_soups only contains soups from alabama and alaska from previous code chunk)
for state_key, city_href in state_city_room_urls.items():
    # add state key to dictionary containers
    state_city_room_responses[state_key] = {}
    state_city_room_soups[state_key] = {}
    
    # loop through urls nested within city href
    for href, room_url in city_href.items():
        # page response and soup containers for city hrefs nested in states
        state_city_room_responses[state_key][href] = []
        state_city_room_soups[state_key][href] = []
        
        # loop through urls and indices, request page responses and make soups from them
        for r, url in enumerate(room_url):
            ## room page response
            room_response_r = requests.get(url)

            ## soup
            room_soup_r = BeautifulSoup(room_response_r.text, "html.parser")

            # add page responses and soup to each container
            # nested dictionaries, e.g., {"alabama": "/auburn": [NEW escape room page response or soup go here, ...]} 
            state_city_room_responses[state_key][href].append(room_response_r)
            state_city_room_soups[state_key][href].append(room_soup_r)

            # sleep for a random number of seconds
            sleep_time_r = np.random.randint(low = 1, high = 10, size = 1)
            time.sleep(sleep_time_r)
        
            # print loop summary
            print("Made soup for {0} in {1}, {2}. Sleeping for {3} seconds...".format(url, href, state_key, sleep_time_j))
        

Made soup for https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-cabin in /auburn, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/auburn/quests/auburn-escape-zones-imprisoned in /auburn, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/auburn/quests/auburn-escape-zones-the-puzzler-vs-superheroes in /auburn, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/auburn/quests/auburn-escape-zones-vault in /auburn, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/auburn/quests/auburn-escape-zones-black-beards-brig in /auburn, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/birm/quests/Locked-In-THE-LABORATORY in /birm, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/birm/quests/locked-in-birm-the-classroom in /birm, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/birm/quests/L

Made soup for https://worldofescapes.com/montgomery/quests/breakout-montgomery-operation-casino in /montgomery, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/montgomery/quests/breakout-games-montgomery-mystery-mansion in /montgomery, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/gulf-shores/quests/Gulf-Coast-Escape-Room-THE-DOLL-ROOM in /gulf-shores, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/gulf-shores/quests/get-a-clue-crime-scene in /gulf-shores, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/gulf-shores/quests/get-a-clue-mardi-gras in /gulf-shores, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/gulf-shores/quests/xit-escape-room-insane-asylum in /gulf-shores, alabama. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/gulf-shores/quests/escape-house-waterville-the-room-of-the-great-mysto in /gulf-shores, 

Made soup for https://worldofescapes.com/phoenix/quests/phoenix-escape-room-gilbert-pipe-works in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/paradox-escape-room-casino-heist in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/epic-escape-game-blood-thirst in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/escape-games-az-aunt-ednas-condo in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/escape-zone-jail-break in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/inferno-escape-room-inventors-secret in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/phoenix-puzzle-room-the-curse-of-madame-lumina in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for ht

Made soup for https://worldofescapes.com/phoenix/quests/dare-to-escape-az-the-shed in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/escape-chandler-the-cabin in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/north-valley-escape-room-the-legend-of-the-7th-stone in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/escape-rooms-mesa-jekyll-and-hyde in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/eludesions-escape-rooms-case-the-wicked-witchs-castle in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/eludesions-escape-rooms-yin-and-yang in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/epic-escape-game-phoenix-the-asylum in /phoenix, arizona. Sleeping for [4] se

Made soup for https://worldofescapes.com/phoenix/quests/riddle-escape-room-dr-riddlers-lab in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/riddle-escape-room-prison-break in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/north-valley-escape-room-cyber-case in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/phoenix/quests/riddle-escape-room-the-heist in /phoenix, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/prescott/quests/prescott-escape-room-dracula-experience in /prescott, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/tucson/quests/escape-room-tucson-area-51 in /tucson, arizona. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/tucson/quests/ace-of-escape-international-thief in /tucson, arizona. Sleeping for [4] seconds...
Made soup for https://world

Made soup for https://worldofescapes.com/fayetteville-ar/quests/the-escapeopolis-project-1969 in /fayetteville-ar, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/fayetteville-ar/quests/the-escapeopolis-project-1959 in /fayetteville-ar, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/fort-smith/quests/great-escape-mystery-rooms-haunted-hotel in /fort-smith, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/fort-smith/quests/fort-smith-escape-room-the-expedition in /fort-smith, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/fort-smith/quests/great-escape-mystery-rooms-prison-break in /fort-smith, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/fort-smith/quests/fort-smith-escape-room-spellbound in /fort-smith, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/fort-smith/quests/fort-smith-escape-room-the-gre

Made soup for https://worldofescapes.com/pine-bluff/quests/escape-room-104-the-marquise-heist in /pine-bluff, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/pine-bluff/quests/escape-room-104-teddys-lab in /pine-bluff, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/russellville/quests/hourglass-escape-rooms-camelot in /russellville, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/russellville/quests/hourglass-escape-rooms-wonderland in /russellville, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/russellville/quests/hourglass-escape-rooms-zombie-pandemic in /russellville, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/russellville/quests/hourglass-escape-rooms-alien in /russellville, arkansas. Sleeping for [4] seconds...
Made soup for https://worldofescapes.com/russellville/quests/hourglass-escape-rooms-alcatraz in /russellville, 

### title

In [349]:
state_city_room_soups["alabama"]["/auburn"][0].find("title").get_text()

'Escape room "The Cabin" by Auburn Escape Zones in Auburn'

### description

In [350]:
state_city_room_soups["alabama"]["/auburn"][0].find("div", {"class": "description"}).get_text()

"Description:You were returning from a trip to your uncle's hunting camp when one of your friends got bitten by a venomous snake, a Timber rattlesnake. There is no cell phone coverage, and each minute counts with your friend's life on the line! You remember that your uncle kept an anti-venom somewhere in his cabin. You have 60 minutes to find it and save your friend."

### address

In [351]:
state_city_room_soups["alabama"]["/auburn"][3].find("div", {"data-content": "address"}).get_text().strip()

'1234 Commerce Dr Auburn, AL 36830 (Show on map)'

### tags

In [352]:
for li in (state_city_room_soups["alabama"]["/auburn"][0]
    .find("div", {"class": "tags"})
    .find("ul", {"class": "tags-2"})
    .find_all("li")):
    print(li.get_text())

Public Ticketing
Up to eight players


In [353]:
state_city_room_soups["alabama"]["/auburn"][0].find("ul", {"class": "params-ul"}).find_all("li")

[<li class="cell" data-content="participants-count"><span class="th">Number of players</span><span class="td">2 – 8</span></li>,
 <li class="cell" data-content="time"><span class="th">Time limit</span><span class="td">60 min.</span></li>,
 <li class="cell" data-content="difficulty"><span class="th">Difficulty level (1-4)</span><span class="td"><span data-original-title="Difficult" data-toggle="tooltip"><i class="fa fa-key"></i><i class="fa fa-key"></i><i class="fa fa-key"></i></span></span></li>,
 <li class="cell"><span class="th">Fear level</span><span class="td scary"><span class="in-words">Not scary</span></span></li>,
 <li class="cell"><span class="th">Age requirement</span><span class="td" data-original-title="Children younger than 13 must be accompanied by an adult." data-toggle="tooltip">10+<sup>*</sup></span></li>,
 <li class="cell"><span class="th">Success rate:</span><span class="td" data-original-title="Success rate:" data-toggle="tooltip"> 54 %</span></li>]

### room attributes

In [354]:
for li in state_city_room_soups["alabama"]["/auburn"][0].find("ul", {"class": "params-ul"}).find_all("li"):
    print(li.find("span", {"class": "td"}).get_text())

2 – 8
60 min.

Not scary
10+*
 54 %


### reviews

In [355]:
for review in state_city_room_soups["alabama"]["/auburn"][0].find("ul", {"class": "masonry-list"}).find_all("p", {"class": "content"}):
    print(review.get_text())

Highly recommended! The staff here are very friendly. We did the Cabin room and had a blast.
We had a spectacular time in the Cabin and saved our friend. The facility was very nice and is adding rooms that we're excited to visit. We'll be back up that way. Definitely worth an hour and a half drive.
Escaped The Cabin and Blackbeard's Brig. Both really well done rooms. I would highly recommend The Cabin! Very friendly staff and excellent facilities!
Auburn Escape Zone is awesome!! I have completed the Cabin, Imprisoned, and tested the new Puzzle master room. All are unique and come with their own challenges. The staff are friendly and are interested in your opinions of the rooms and are constantly thinking of new and creative rooms. If you want a challenge and a fun time I HIGHLY suggest you give one of the rooms a try.
I now have played all of the rooms at Auburn Escape Zones. As a huge fan of escape rooms, I can tell you this is top notch. Their puzzles and tasks are certainly challeng

## Escape room/game titles, descriptions, and addresses

In [356]:
# containers for titles, descriptions, and addresses
state_city_room_titles = {}
state_city_room_descriptions = {}
state_city_room_addresses = {}

# loop through city href nested within state keys
# just alabama and alaska for now (state_city_soups only contains soups from alabama and alaska from previous code chunk)
for state_key, city_href in state_city_room_soups.items():
    # add state key to dictionary containers
    state_city_room_titles[state_key] = {}
    state_city_room_descriptions[state_key] = {}
    state_city_room_addresses[state_key] = {}
    
    # loop through room soups nested within city href
    for href, room_soup in city_href.items():
        # room containers for city hrefs nested in states
        state_city_room_titles[state_key][href] = {}
        state_city_room_descriptions[state_key][href] = {}
        state_city_room_addresses[state_key][href] = {}
    
        # loop through soups nested within city href
        for k, room in enumerate(room_soup):
            # room containers for city href nested in states
            # e.g. {"alabama": {"/auburn": {"/auburn/quests/auburn-escape-zones-imprisoned": [review_1, ...]}}}
            state_city_room_titles[state_key][href][state_city_room_href[state_key][href][k]] = []
            state_city_room_descriptions[state_key][href][state_city_room_href[state_key][href][k]] = []
            state_city_room_addresses[state_key][href][state_city_room_href[state_key][href][k]] = []
            
            # title, description, address
            title_k = room.find("title").get_text()
            description_k = room.find("div", {"class": "description"}).get_text()
            address_k = room.find("div", {"data-content": "address"}).get_text().strip()
            
            # add title, description, and address to their containers
            state_city_room_titles[state_key][href][state_city_room_href[state_key][href][k]].append(title_k)
            state_city_room_descriptions[state_key][href][state_city_room_href[state_key][href][k]].append(description_k)
            state_city_room_addresses[state_key][href][state_city_room_href[state_key][href][k]].append(address_k)
            
            # print loop summary
            print("{0}: {1} is located at {2}.\n{3}: {4}.".format(k, title_k, address_k, k, description_k))

0: Escape room "The Cabin" by Auburn Escape Zones in Auburn is located at 1234 Commerce Dr Auburn, AL 36830 (Show on map).
0: Description:You were returning from a trip to your uncle's hunting camp when one of your friends got bitten by a venomous snake, a Timber rattlesnake. There is no cell phone coverage, and each minute counts with your friend's life on the line! You remember that your uncle kept an anti-venom somewhere in his cabin. You have 60 minutes to find it and save your friend..
1: Escape room "Imprisoned" by Auburn Escape Zones in Auburn is located at 1234 Commerce Dr Auburn, AL 36830 (Show on map).
1: Description:Your friend is being framed for a crime he didn't commit! He's held in a corrupt prison, and the warden has bribed the other inmates to hide the evidence needed to exonerate your friend. Determined to make things right, you've managed to sneak into the warden's office. With just an hour left before the death row, can you find the necessary proof to clear your fri

1: Escape room "Espionage" by Escape Room Florence in Florence (AL) is located at 108 S. Pine St., Florence, AL 35630 (Show on map).
1: Description:As a veteran American spy, you’ve been hired to work with a longtime British MI6 colleague Kim Philby. As you near the heart of the Soviet weapon plans, you feel a gun to your back. It turns out, you’ve been betrayed and Kim is actually a double agent working for the KGB.
He’s handed you over and the Russians are planning a very intense interrogation for you. Handcuffed and with only one hour before they come back, you must find your way out of this holding cell. You don’t want to find out what they do to extract information from spies!
This room is not handicap accessible. You may have to lift up to 10 lbs, squat, crawl or kneel..
2: Escape room "Sound Escape" by Escape Room Florence in Florence (AL) is located at 108 S. Pine St., Florence, AL 35630 (Show on map).
2: Description:You're a local producer in a city that has claimed to be a hi

.
7: Escape room "Antidote" by Escapology Huntsville in Huntsville is located at 930 Old Monrovia Rd NW, Suite A, Huntsville AL 35806 (Show on map).
7: Description:Your research group was invited to gather evidence in Dr. Brandt's lab. He has been working for the enemy to develop a deadly virus that could eliminate thousands of people. Locked in his lab, you have one hour to save your lives and the entire world from destructive chemical warfare. The clock is ticking!.
8: Escape room "Under Pressure" by Escapology Huntsville in Huntsville is located at 930 Old Monrovia Rd NW, Suite A, Huntsville AL 35806 (Show on map).
8: Description:The year is 1944: the height of World War II. Onboard the Steel Shark, deep in the North Atlantic Ocean, your team is on a mission to spy on the German Battle Cruiser, Scheer. Suddenly, sabotaged by German agents, the submarine goes into emergency mode and only 60 minutes of reserve oxygen remains! You and the crew must act quickly so you can stay alive and

0: Description:There once was a little girl who was very attached to her dolls. One day, her toys had enough and took her life in a series of mysterious and unexplained events. The girl is no longer with us, but her spirit was left behind. Can you help her find peace and escape the room before the dolls decide to take you as their forever prisoner?.
1: Escape room "Crime Scene" by Get a Clue in Orange Beach is located at 1125 N McKenzie St Foley, AL 36535 (Show on map).
1: Description:A DEA agent who was investigating a case regarding a mafia drug business was killed. Before he was attacked, the agent called to inform you of new leads he had found. Do you and your team have what it takes to find his assailant and the hidden evidence before time runs out?.
2: Escape room "Mardi Gras" by Get a Clue in Orange Beach is located at 1125 N McKenzie St Foley, AL 36535 (Show on map).
2: Description:It’s carnival season in Downtown Mobile. You and your friends are in search of a party when you e

Are you up for the challenge?.
4: Escape room "Saloon No 10" by Escape Anchorage in Anchorage is located at 20905 Eastside Dr Chugiak, AK 99567 (Show on map).
4: Description:The saloon is loaded with cash and valuables but someone has rigged the place to explode. Get as much loot as you can take with you and get out before the bomb detonates! 

.
5: Escape room "Sanatorium" by Escape Anchorage in Anchorage is located at 20905 Eastside Dr Chugiak, AK 99567 (Show on map).
5: Description:Welcome to Smith's Grove, a sanitarium for the crimeanly insane. Lucky for you it was abandoned years ago and reutilized as a psychological testing ground. Do you have what it takes to navigate your way out of this observation room?.
6: Escape room "It's Mine" by Avalanche Escape Rooms in Anchorage is located at 4111 Minnesota Drive, Anchorage, AK 99503 (Show on map).
6: Description:Abandon for nearly a century, the decrepit gold mine has been located. Can the wealth be recovered, or will it be buried for

...

117: Escape room "Disco Fever!" by Escape Rooms Mesa in Phoenix is located at 86 W University Dr #100, Mesa, AZ 85201 (Show on map).
117: Description:You were invited to the disco. Is it business or pleasure? Will you be the victim or the investigators. Be careful the zodiac killer is still out there, and you might be his next dance!.
118: Escape room "The Dig" by Escape The Room Scottsdale in Phoenix is located at 7017 E. Main St Scottsdale, AZ 85251 (Show on map).
118: Description:A mixture of Indiana Jones and Da Vinci Code, the Dig is a multi-layered game with hidden twists and unexpected turns. Can you solve the mystery and escape?.
119: Escape room "Jurassic Escape" by Escape The Room Scottsdale in Phoenix is located at 7017 E Main St Scottsdale, AZ 85251 (Show on map).
119: Description:Dinosaurs are loose and hunting again in Oak Brook thanks to an evil corporation’s cloning experiments. It’s up to you and your team to stop their cloning process as you sneak through their facili

19: Description:After discovering a new tomb deep under the pyramids, workers started to disappear.  Now, many are frightened of what might be lurking in the cold darkness.  They say the tomb is cursed!  If it really is cursed, then it is up to you to put a stop to all the madness and find the ancient artifacts that will keep the dead, dead, and keep you, the living, alive. .
20: Escape room "Human Xperiment" by Xcape House in Tucson is located at 1102 W Grant Rd Tucson, AZ 85705 (Show on map).
20: Description:Each team will have a 1 hour window of time while the Doctor is away to try to break out by using clues left behind by previous subjects and some of the Doctor's assistants who have realized his wrong doings but are too afraid to alert the authorities. By using the clues left behind by other subjects, the teams will be able to piece together the secrets to their escape. Some clues are subtle, some are obvious but all play a roll and using every member of the team will be essentia

7: Description:You've infiltrated the office of a notorious arms dealer. Can you find the evidence and escape before you're caught?.
8: Escape room "Santa's Workshop Head-to-Head Adventure" by NWA Escape Room in Fayetteville (AR) is located at 4083 North Shiloh Drive, Fayetteville, AR 72703 (Show on map).
8: Description:The Grinch is Back! Santa's workshop has been sabotaged and only you can save the day and keep all the girls and boys from being disappointed on Christmas! Can you think outside the box and use all your wits to undo the Grinch's scheme, find the missing toys and save Christmas?.
9: Escape room "1969 - Insurgent Attack" by The Escapeopolis Project in Fayetteville (AR) is located at 406 East Henri De Tonti Boulevard Tontitown, AR 72762 (Show on map).
9: Description:Your team of new recruits has been assigned to the Security Operations (SecOps) center of The Escapeopolis Project. SecOps is responsible for the security of the entire Escapeopolis facility and it's Thorium-Fu

1: Escape room "The Psych Ward" by Locked Away Escapes in Jonesboro is located at 3226 Shelby Dr Jonesboro, AR 72404 (Show on map).
1: Description:You are committed to a mental institution by your loved ones. You are not crazy. You just chose the wrong enemy, Anna, a deceitful menacing woman who hides under a false veil of charitable innocence. She plots murder and needs you out of the way before you blow her cover. Your warnings about Anna fall on deaf ears, and in turn, you are committed. Will you find the way out of your cell and expose Anna, or will you forever be in solitary confinement?.
2: Escape room "The Attic" by Locked Away Escapes in Jonesboro is located at 3226 Shelby Dr Jonesboro, AR 72404 (Show on map).
2: Description:A distant aunt has recently passed, leaving your family to deal with her vast estate's welfare. You've decided to sell it to the city and the demolition of the house is scheduled to take place in an hour. You've collected all the stuff from the manor, but y

17: Escape room "ELF" by Central Arkansas Escape Rooms in Little Rock is located at 108 West F Avenue North Little Rock, AR 72116 (Show on map).
17: Description:It's Christmas time at Central Arkansas Escape Rooms! But wait ... Santa's elves are on strike! The sleigh won't fly, and orders are mixed up! Christmas may have to be cancelled, unless ... YOU assemble your own team of elves. Find the secret entrance to Santa's Workshop and GET. TO. WORK! Can you save Christmas!? There's only one way to find out! .
18: Escape room "Pirate's Plunder" by Arkansas Escape Rooms in Little Rock is located at 2615 North Prickett Road, Suite 4 Bryant, AR 72022 (Show on map).
18: Description:Welcome aboard this old clipper ship, owned by the eccentric John Buckingham. He has recently taken to acting, talking and dressing like a pirate, and claims that somewhere inside the ship lies the most valuable treasure ever. You will have 60 minutes to try and find it..
19: Escape room "Battleship" by Red Door Es

### Store room/game titles, descriptions, and addresses in a data frame

In [357]:
state_city_room_titles

{'alabama': {'/auburn': {'/auburn/quests/auburn-escape-zones-the-cabin': ['Escape room "The Cabin" by Auburn Escape Zones in Auburn'],
   '/auburn/quests/auburn-escape-zones-imprisoned': ['Escape room "Imprisoned" by Auburn Escape Zones in Auburn'],
   '/auburn/quests/auburn-escape-zones-the-puzzler-vs-superheroes': ['Escape room "The Puzzler vs Superheroes" by Auburn Escape Zones in Auburn'],
   '/auburn/quests/auburn-escape-zones-vault': ['Escape room "Vault" by Auburn Escape Zones in Auburn'],
   '/auburn/quests/auburn-escape-zones-black-beards-brig': ['Escape room "Black Beard\'s Brig" by Auburn Escape Zones in Auburn']},
  '/birm': {'/birm/quests/Locked-In-THE-LABORATORY': ['Escape room "The Laboratory" by Locked In in Birmingham'],
   '/birm/quests/locked-in-birm-the-classroom': ['Escape room "The Classroom" by Locked In in Birmingham'],
   '/birm/quests/Locked-In-THE-WAREHOUSE': ['Escape room "The Warehouse" by Locked In in Birmingham'],
   '/birm/quests/nightmare-at-3008-3008-h

In [358]:
# titles
## data frame container
escape_room_titles = pd.DataFrame(columns = {"state", "city_href", "room_href", "room_title"})

# loop through city href nested in state keys
for state, city_href in state_city_room_titles.items():
    # loop through room href nested within city href keys
    for city, room_href in city_href.items():
        # loop through review lists nested in room href keys
        for room, titles in room_href.items():
            # loop through titles (there is only 1)
            for title in titles:
                
                # append to data frame (append does not occur in place, so need to assign appended data frame)
                escape_room_titles = escape_room_titles.append(pd.DataFrame({"state": state, "city_href": city, "room_href": room, "room_title": titles}, index = [0]), ignore_index = True)
                
# descriptions
## data frame container
escape_room_descriptions = pd.DataFrame(columns = {"state", "city_href", "room_href", "room_description"})

# loop through city href nested in state keys
for state, city_href in state_city_room_descriptions.items():
    # loop through room href nested within city href keys
    for city, room_href in city_href.items():
        # loop through description lists nested in room href keys
        for room, descriptions in room_href.items():
            # loop through descriptions (there is only 1)
            for description in descriptions:
                
                # append to data frame (append does not occur in place, so need to assign appended data frame)
                escape_room_descriptions = escape_room_descriptions.append(pd.DataFrame({"state": state, "city_href": city, "room_href": room, "room_description": description}, index = [0]), ignore_index = True)

# addresses
## data frame container
escape_room_addresses = pd.DataFrame(columns = {"state", "city_href", "room_href", "room_address"})

# loop through city href nested in state keys
for state, city_href in state_city_room_addresses.items():
    # loop through room href nested within city href keys
    for city, room_href in city_href.items():
        # loop through address lists nested in room href keys
        for room, addresses in room_href.items():
            # loop through addresses (there is only 1)
            for address in addresses:
                
                # append to data frame (append does not occur in place, so need to assign appended data frame)
                escape_room_addresses = escape_room_addresses.append(pd.DataFrame({"state": state, "city_href": city, "room_href": room, "room_address": address}, index = [0]), ignore_index = True)
                
# merge data frames
escape_room_info = (escape_room_titles
                    .merge(escape_room_descriptions, how = "left", on = ["state", "city_href", "room_href"])
                    .merge(escape_room_addresses, how = "left", on = ["state", "city_href", "room_href"]))

# See it
escape_room_info

,room_title,state,room_href,city_href,room_description,room_address
0,"Escape room ""The Cabin"" by Auburn Escape Zones...",alabama,/auburn/quests/auburn-escape-zones-the-cabin,/auburn,Description:You were returning from a trip to ...,"1234 Commerce Dr Auburn, AL 36830 (Show on map)"
1,"Escape room ""Imprisoned"" by Auburn Escape Zone...",alabama,/auburn/quests/auburn-escape-zones-imprisoned,/auburn,Description:Your friend is being framed for a ...,"1234 Commerce Dr Auburn, AL 36830 (Show on map)"
2,"Escape room ""The Puzzler vs Superheroes"" by Au...",alabama,/auburn/quests/auburn-escape-zones-the-puzzler...,/auburn,Description:The Puzzler is up to his old trick...,"1234 Commerce Dr Auburn, AL 36830 (Show on map)"
3,"Escape room ""Vault"" by Auburn Escape Zones in ...",alabama,/auburn/quests/auburn-escape-zones-vault,/auburn,Description:It’s Grand Opening day at the new ...,"1234 Commerce Dr Auburn, AL 36830 (Show on map)"
4,"Escape room ""Black Beard's Brig"" by Auburn Esc...",alabama,/auburn/quests/auburn-escape-zones-black-beard...,/auburn,Description:Ahoy Matey! Your crew has been cap...,"1234 Commerce Dr Auburn, AL 36830 (Show on map)"
...,...,...,...,...,...,...
351,"Escape room ""Asylum"" by Hourglass Escape Rooms...",arkansas,/russellville/quests/hourglass-escape-rooms-as...,/russellville,Description:While exploring an old insane asyl...,"2606 West Main Street Russellville, AR 72801 (..."
352,"Escape room ""Asylum"" by Hourglass Escape Rooms...",arkansas,/searcy/quests/hourglass-escape-rooms-searcy-a...,/searcy,Description: While exploring an old insane asy...,"1308 E Race Ave Searcy, AR 72143 (Show on map)"
353,"Escape room ""Alcatraz"" by Hourglass Escape Roo...",arkansas,/searcy/quests/hourglass-escape-rooms-searcy-a...,/searcy,Description:Your crew has been thrown into the...,"1308 E Race Ave Searcy, AR 72143 (Show on map)"
354,"Escape room ""Wonderland"" by Hourglass Escape R...",arkansas,/searcy/quests/hourglass-escape-rooms-searcy-w...,/searcy,Description: After following the white rabb...,"1308 E Race Ave Searcy, AR 72143 (Show on map)"


## Escape room/game reviews

In [359]:
# container for reviews nested within game nested within city nested within state
state_city_room_reviews = {}

# loop through city href nested within states
# just alabama and alaska for now (state_city_soups only contains soups from alabama and alaska from previous code chunk)
for state_key, city_href in state_city_room_soups.items():
    # add state key to dictionary containers
    state_city_room_reviews[state_key] = {}
    
    # loop through urls nested within city href
    for href, room_soup in city_href.items():
        # room containers for city hrefs nested in states
        state_city_room_reviews[state_key][href] = {}
    
        # loop through urls nested within city href
        for k, room in enumerate(room_soup):
            # room containers for city href nested in states
            # e.g. {"alabama": {"/auburn": {"/auburn/quests/auburn-escape-zones-imprisoned": [review_1, ...]}}}
            state_city_room_reviews[state_key][href][state_city_room_href[state_key][href][k]] = []
            
            # review html
            if room.find("ul", {"class": "masonry-list"}) is not None:
                review_content = room.find("ul", {"class": "masonry-list"}).find_all("p", {"class": "content"})
            
                # loop through review content
                for r, review in enumerate(review_content, start = 1):
                    state_city_room_reviews[state_key][href][state_city_room_href[state_key][href][k]].append(review.get_text())

                    # print loop summary
                    print("Scraped {0} out of {1} reviews of {2} in {3}, {4}.".format(r, len(review_content), state_city_room_href[state_key][href][k], href, state_key, sleep_time_j))
            else:
                state_city_room_reviews[state_key][href][state_city_room_href[state_key][href][k]].append("None")
                print("No reviews for {0} in {1}, {2}.".format(state_city_room_href[state_key][href][k], href, state_key, sleep_time_j))

Scraped 1 out of 5 reviews of /auburn/quests/auburn-escape-zones-the-cabin in /auburn, alabama.
Scraped 2 out of 5 reviews of /auburn/quests/auburn-escape-zones-the-cabin in /auburn, alabama.
Scraped 3 out of 5 reviews of /auburn/quests/auburn-escape-zones-the-cabin in /auburn, alabama.
Scraped 4 out of 5 reviews of /auburn/quests/auburn-escape-zones-the-cabin in /auburn, alabama.
Scraped 5 out of 5 reviews of /auburn/quests/auburn-escape-zones-the-cabin in /auburn, alabama.
Scraped 1 out of 5 reviews of /auburn/quests/auburn-escape-zones-imprisoned in /auburn, alabama.
Scraped 2 out of 5 reviews of /auburn/quests/auburn-escape-zones-imprisoned in /auburn, alabama.
Scraped 3 out of 5 reviews of /auburn/quests/auburn-escape-zones-imprisoned in /auburn, alabama.
Scraped 4 out of 5 reviews of /auburn/quests/auburn-escape-zones-imprisoned in /auburn, alabama.
Scraped 5 out of 5 reviews of /auburn/quests/auburn-escape-zones-imprisoned in /auburn, alabama.
Scraped 1 out of 3 reviews of /aubu

Scraped 1 out of 3 reviews of /montgomery/quests/breakout-games-montgomery-mystery-mansion in /montgomery, alabama.
Scraped 2 out of 3 reviews of /montgomery/quests/breakout-games-montgomery-mystery-mansion in /montgomery, alabama.
Scraped 3 out of 3 reviews of /montgomery/quests/breakout-games-montgomery-mystery-mansion in /montgomery, alabama.
Scraped 1 out of 5 reviews of /gulf-shores/quests/Gulf-Coast-Escape-Room-THE-DOLL-ROOM in /gulf-shores, alabama.
Scraped 2 out of 5 reviews of /gulf-shores/quests/Gulf-Coast-Escape-Room-THE-DOLL-ROOM in /gulf-shores, alabama.
Scraped 3 out of 5 reviews of /gulf-shores/quests/Gulf-Coast-Escape-Room-THE-DOLL-ROOM in /gulf-shores, alabama.
Scraped 4 out of 5 reviews of /gulf-shores/quests/Gulf-Coast-Escape-Room-THE-DOLL-ROOM in /gulf-shores, alabama.
Scraped 5 out of 5 reviews of /gulf-shores/quests/Gulf-Coast-Escape-Room-THE-DOLL-ROOM in /gulf-shores, alabama.
Scraped 1 out of 5 reviews of /gulf-shores/quests/get-a-clue-crime-scene in /gulf-shore

Scraped 1 out of 5 reviews of /phoenix/quests/north-valley-escape-room-escape-from-alcatraz in /phoenix, arizona.
Scraped 2 out of 5 reviews of /phoenix/quests/north-valley-escape-room-escape-from-alcatraz in /phoenix, arizona.
Scraped 3 out of 5 reviews of /phoenix/quests/north-valley-escape-room-escape-from-alcatraz in /phoenix, arizona.
Scraped 4 out of 5 reviews of /phoenix/quests/north-valley-escape-room-escape-from-alcatraz in /phoenix, arizona.
Scraped 5 out of 5 reviews of /phoenix/quests/north-valley-escape-room-escape-from-alcatraz in /phoenix, arizona.
Scraped 1 out of 5 reviews of /phoenix/quests/phoenix-escape-room-gilbert-pipe-works in /phoenix, arizona.
Scraped 2 out of 5 reviews of /phoenix/quests/phoenix-escape-room-gilbert-pipe-works in /phoenix, arizona.
Scraped 3 out of 5 reviews of /phoenix/quests/phoenix-escape-room-gilbert-pipe-works in /phoenix, arizona.
Scraped 4 out of 5 reviews of /phoenix/quests/phoenix-escape-room-gilbert-pipe-works in /phoenix, arizona.
Sc

Scraped 1 out of 3 reviews of /phoenix/quests/alcatraz-escape-games-horcrux-hysteria in /phoenix, arizona.
Scraped 2 out of 3 reviews of /phoenix/quests/alcatraz-escape-games-horcrux-hysteria in /phoenix, arizona.
Scraped 3 out of 3 reviews of /phoenix/quests/alcatraz-escape-games-horcrux-hysteria in /phoenix, arizona.
Scraped 1 out of 3 reviews of /phoenix/quests/escape-games-az-the-bake-off in /phoenix, arizona.
Scraped 2 out of 3 reviews of /phoenix/quests/escape-games-az-the-bake-off in /phoenix, arizona.
Scraped 3 out of 3 reviews of /phoenix/quests/escape-games-az-the-bake-off in /phoenix, arizona.
Scraped 1 out of 3 reviews of /phoenix/quests/phoenix-escape-room-the-crimson-storm in /phoenix, arizona.
Scraped 2 out of 3 reviews of /phoenix/quests/phoenix-escape-room-the-crimson-storm in /phoenix, arizona.
Scraped 3 out of 3 reviews of /phoenix/quests/phoenix-escape-room-the-crimson-storm in /phoenix, arizona.
Scraped 1 out of 3 reviews of /phoenix/quests/escape-zone-az-the-islan

Scraped 1 out of 1 reviews of /phoenix/quests/eludesions-escape-rooms-the-painters-collection in /phoenix, arizona.
Scraped 1 out of 7 reviews of /phoenix/quests/lockdown-escape-rooms-the-theft in /phoenix, arizona.
Scraped 2 out of 7 reviews of /phoenix/quests/lockdown-escape-rooms-the-theft in /phoenix, arizona.
Scraped 3 out of 7 reviews of /phoenix/quests/lockdown-escape-rooms-the-theft in /phoenix, arizona.
Scraped 4 out of 7 reviews of /phoenix/quests/lockdown-escape-rooms-the-theft in /phoenix, arizona.
Scraped 5 out of 7 reviews of /phoenix/quests/lockdown-escape-rooms-the-theft in /phoenix, arizona.
Scraped 6 out of 7 reviews of /phoenix/quests/lockdown-escape-rooms-the-theft in /phoenix, arizona.
Scraped 7 out of 7 reviews of /phoenix/quests/lockdown-escape-rooms-the-theft in /phoenix, arizona.
Scraped 1 out of 6 reviews of /phoenix/quests/escape-rooms-mesa-mad-hatters-tea-party in /phoenix, arizona.
Scraped 2 out of 6 reviews of /phoenix/quests/escape-rooms-mesa-mad-hatters-

Scraped 1 out of 1 reviews of /tucson/quests/escape-room-tucson-mysteries-of-egypt in /tucson, arizona.
Scraped 1 out of 1 reviews of /tucson/quests/will-you-escape-the-witching-hour in /tucson, arizona.
Scraped 1 out of 5 reviews of /tucson/quests/escape-room-tucson-bomb-squad in /tucson, arizona.
Scraped 2 out of 5 reviews of /tucson/quests/escape-room-tucson-bomb-squad in /tucson, arizona.
Scraped 3 out of 5 reviews of /tucson/quests/escape-room-tucson-bomb-squad in /tucson, arizona.
Scraped 4 out of 5 reviews of /tucson/quests/escape-room-tucson-bomb-squad in /tucson, arizona.
Scraped 5 out of 5 reviews of /tucson/quests/escape-room-tucson-bomb-squad in /tucson, arizona.
No reviews for /tucson/quests/fox-in-a-box-tucson-wild-west in /tucson, arizona.
No reviews for /tucson/quests/mystery-escape-room-tucson-20000-leagues-under-the-sea in /tucson, arizona.
No reviews for /tucson/quests/mystery-escape-room-tucson-sherlock-holmes-moriartys-parlor in /tucson, arizona.
No reviews for /tu

Scraped 1 out of 13 reviews of /fayetteville-ar/quests/the-escapeopolis-project-1959 in /fayetteville-ar, arkansas.
Scraped 2 out of 13 reviews of /fayetteville-ar/quests/the-escapeopolis-project-1959 in /fayetteville-ar, arkansas.
Scraped 3 out of 13 reviews of /fayetteville-ar/quests/the-escapeopolis-project-1959 in /fayetteville-ar, arkansas.
Scraped 4 out of 13 reviews of /fayetteville-ar/quests/the-escapeopolis-project-1959 in /fayetteville-ar, arkansas.
Scraped 5 out of 13 reviews of /fayetteville-ar/quests/the-escapeopolis-project-1959 in /fayetteville-ar, arkansas.
Scraped 6 out of 13 reviews of /fayetteville-ar/quests/the-escapeopolis-project-1959 in /fayetteville-ar, arkansas.
Scraped 7 out of 13 reviews of /fayetteville-ar/quests/the-escapeopolis-project-1959 in /fayetteville-ar, arkansas.
Scraped 8 out of 13 reviews of /fayetteville-ar/quests/the-escapeopolis-project-1959 in /fayetteville-ar, arkansas.
Scraped 9 out of 13 reviews of /fayetteville-ar/quests/the-escapeopolis-

No reviews for /pine-bluff/quests/escape-room-104-a-diamond-dilemma in /pine-bluff, arkansas.
No reviews for /pine-bluff/quests/escape-room-104-the-marquise-heist in /pine-bluff, arkansas.
No reviews for /pine-bluff/quests/escape-room-104-teddys-lab in /pine-bluff, arkansas.
Scraped 1 out of 3 reviews of /russellville/quests/hourglass-escape-rooms-camelot in /russellville, arkansas.
Scraped 2 out of 3 reviews of /russellville/quests/hourglass-escape-rooms-camelot in /russellville, arkansas.
Scraped 3 out of 3 reviews of /russellville/quests/hourglass-escape-rooms-camelot in /russellville, arkansas.
Scraped 1 out of 3 reviews of /russellville/quests/hourglass-escape-rooms-wonderland in /russellville, arkansas.
Scraped 2 out of 3 reviews of /russellville/quests/hourglass-escape-rooms-wonderland in /russellville, arkansas.
Scraped 3 out of 3 reviews of /russellville/quests/hourglass-escape-rooms-wonderland in /russellville, arkansas.
Scraped 1 out of 1 reviews of /russellville/quests/hour

In [360]:
state_city_room_reviews

{'alabama': {'/auburn': {'/auburn/quests/auburn-escape-zones-the-cabin': ['Highly recommended! The staff here are very friendly. We did the Cabin room and had a blast.',
    "We had a spectacular time in the Cabin and saved our friend. The facility was very nice and is adding rooms that we're excited to visit. We'll be back up that way. Definitely worth an hour and a half drive.",
    "Escaped The Cabin and Blackbeard's Brig. Both really well done rooms. I would highly recommend The Cabin! Very friendly staff and excellent facilities!",
    'Auburn Escape Zone is awesome!! I have completed the Cabin, Imprisoned, and tested the new Puzzle master room. All are unique and come with their own challenges. The staff are friendly and are interested in your opinions of the rooms and are constantly thinking of new and creative rooms. If you want a challenge and a fun time I HIGHLY suggest you give one of the rooms a try.',
    "I now have played all of the rooms at Auburn Escape Zones. As a hug

### Store reviews in a data frame

In [361]:
# data frame container
escape_room_reviews = pd.DataFrame(columns = ["state", "city_href", "room_href", "review_number", "review"])

# loop through city href nested in state keys
for state, city_href in state_city_room_reviews.items():
    # loop through room href nested within city href keys
    for city, room_href in city_href.items():
        # loop through review lists nested in room href keys
        for room, reviews in room_href.items():
            # loop through reviews
            for r, review in enumerate(reviews):
                # append to data frame (append does not occur in place, so need to assign appended data frame)
                escape_room_reviews = escape_room_reviews.append(pd.DataFrame({"state": state, "city_href": city, "room_href": room, "review_number": r, "review": review}, index = [0]), ignore_index = True)

# See it
escape_room_reviews

,state,city_href,room_href,review_number,review
0,alabama,/auburn,/auburn/quests/auburn-escape-zones-the-cabin,0,Highly recommended! The staff here are very fr...
1,alabama,/auburn,/auburn/quests/auburn-escape-zones-the-cabin,1,We had a spectacular time in the Cabin and sav...
2,alabama,/auburn,/auburn/quests/auburn-escape-zones-the-cabin,2,Escaped The Cabin and Blackbeard's Brig. Both ...
3,alabama,/auburn,/auburn/quests/auburn-escape-zones-the-cabin,3,Auburn Escape Zone is awesome!! I have complet...
4,alabama,/auburn,/auburn/quests/auburn-escape-zones-the-cabin,4,I now have played all of the rooms at Auburn E...
...,...,...,...,...,...
1022,arkansas,/searcy,/searcy/quests/hourglass-escape-rooms-searcy-a...,2,Escaped from Alcatraz with Lauren and Mike! Aw...
1023,arkansas,/searcy,/searcy/quests/hourglass-escape-rooms-searcy-w...,0,This place was an absolute blast!! Went to try...
1024,arkansas,/searcy,/searcy/quests/hourglass-escape-rooms-searcy-w...,1,Sugar and I had so much fun in the Alice in Wo...
1025,arkansas,/searcy,/searcy/quests/hourglass-escape-rooms-searcy-w...,2,We did the Alice in Wonderland room. It was gr...


### Merge reviews with escape room/game info

In [362]:
# merge data frames
escape_room_reviews = escape_room_reviews.merge(escape_room_info, how = "left", on = ["state", "city_href", "room_href"])

# See it
escape_room_reviews

,state,city_href,room_href,review_number,review,room_title,room_description,room_address
0,alabama,/auburn,/auburn/quests/auburn-escape-zones-the-cabin,0,Highly recommended! The staff here are very fr...,"Escape room ""The Cabin"" by Auburn Escape Zones...",Description:You were returning from a trip to ...,"1234 Commerce Dr Auburn, AL 36830 (Show on map)"
1,alabama,/auburn,/auburn/quests/auburn-escape-zones-the-cabin,1,We had a spectacular time in the Cabin and sav...,"Escape room ""The Cabin"" by Auburn Escape Zones...",Description:You were returning from a trip to ...,"1234 Commerce Dr Auburn, AL 36830 (Show on map)"
2,alabama,/auburn,/auburn/quests/auburn-escape-zones-the-cabin,2,Escaped The Cabin and Blackbeard's Brig. Both ...,"Escape room ""The Cabin"" by Auburn Escape Zones...",Description:You were returning from a trip to ...,"1234 Commerce Dr Auburn, AL 36830 (Show on map)"
3,alabama,/auburn,/auburn/quests/auburn-escape-zones-the-cabin,3,Auburn Escape Zone is awesome!! I have complet...,"Escape room ""The Cabin"" by Auburn Escape Zones...",Description:You were returning from a trip to ...,"1234 Commerce Dr Auburn, AL 36830 (Show on map)"
4,alabama,/auburn,/auburn/quests/auburn-escape-zones-the-cabin,4,I now have played all of the rooms at Auburn E...,"Escape room ""The Cabin"" by Auburn Escape Zones...",Description:You were returning from a trip to ...,"1234 Commerce Dr Auburn, AL 36830 (Show on map)"
...,...,...,...,...,...,...,...,...
1022,arkansas,/searcy,/searcy/quests/hourglass-escape-rooms-searcy-a...,2,Escaped from Alcatraz with Lauren and Mike! Aw...,"Escape room ""Alcatraz"" by Hourglass Escape Roo...",Description:Your crew has been thrown into the...,"1308 E Race Ave Searcy, AR 72143 (Show on map)"
1023,arkansas,/searcy,/searcy/quests/hourglass-escape-rooms-searcy-w...,0,This place was an absolute blast!! Went to try...,"Escape room ""Wonderland"" by Hourglass Escape R...",Description: After following the white rabb...,"1308 E Race Ave Searcy, AR 72143 (Show on map)"
1024,arkansas,/searcy,/searcy/quests/hourglass-escape-rooms-searcy-w...,1,Sugar and I had so much fun in the Alice in Wo...,"Escape room ""Wonderland"" by Hourglass Escape R...",Description: After following the white rabb...,"1308 E Race Ave Searcy, AR 72143 (Show on map)"
1025,arkansas,/searcy,/searcy/quests/hourglass-escape-rooms-searcy-w...,2,We did the Alice in Wonderland room. It was gr...,"Escape room ""Wonderland"" by Hourglass Escape R...",Description: After following the white rabb...,"1308 E Race Ave Searcy, AR 72143 (Show on map)"


### Export escape room reviews data to .csv

In [363]:
escape_room_reviews.to_csv("data/escape_room_reviews.csv")